<font size="5">Raw Version</font>

Setting up probes:

In [3]:
# desired DNA strand
MBP_seq = "AAAGACAGGCCCTCAGAGTCCGACGAGCTTCTCAGAGTCCGACGAGCTTCAGACCATCCAAGAAGATCCCACAGCAGCTTCCGAAGGCCTGGATGTGATGGCATCACAGAAGAGACCCTCACAGCGACACGGGTCCAAGTACTTGGCCACAGCAAGTACCATGGACCATGCCCGGCATGGCTTCCTCCCAAGGCACAGAGACACGGGCATCCTTGACTCCATCGGGCGCTTCTTTAGCGGTGACAGGGGTGCGCCCAAGCGGGGCTCTGGCAAGGTACCCTGGCTAAAGCAGAGCCGGAGCCCTCTGCCTTCTCATGCCCGCAGCCGTCCCGGGCTGTGCCACATGTACAAGGACTCACACACAAGAACTACCCACTACGGCTCCCTGCCCCAGAAGTCGCAGAGGACCCAAGATGAAAACCCAGTAGTCCACTTCTTCAAGAACATTGTGACACCTCGTACACCCCCTCCATCCCAAGGAAAGGGGAGAGGCCTGTCCCTCAGCAGATTTAGCTGGGGGGCCGAGGGGCAGAAGCCAGGATTTGGCTACGGAGGCAGAGCTTCCGACTATAAATCGGCTCACAAGGGATTCAAGGGGGCCTACGACGCCCAGGGCACGCTTTCCAAAATCTTTAAGCTGGGAGGAAGAGACAGCCGCTCTGGATCTCCCATGGCAAGACGCTGAGAGCCTCCCTGCTCAGCCTTCCCGAATCCTGCCCTCGGCTTCTTAATATAACTGCCTTAAACGTTTAATTCTACTTGCACCAAATAGCTAGTTAGAGCAGACCCTCTCTTAATCCCGTGGGGCTGTGAACGCGGCGGGGCCAGGCCCACGGCACCCTGACTGGCTAAAACTGTTTGTCCCTTTTTATTTGAAGATTGAGTTTCCTCGGGGTCTTCTCTGCCCCGACTTGCTCCCCGTGTACCTTGGTCGACTCCGGAGGTTCAGGTGCACGGACACCCTTTCAAGTTCACCCCTACTCCATCCTCAGACTTTCTTTTCACGGCGAGGCGCACCCCTCCAGCTTCCGTGGGCACTGCGGATAGACAGGCACACCGCCAAGGAGCCAGAGAGCATGGCGCAGGGGACTGTGTGGTCCAGGCTTCCTTTGTTTTCTTTCCCCTAAAGAGCTTTGTTTTTCCTAACAGGATCAGACAGTCTTGGAGTGGCTTACACAACGGGGGCTTGTGGTATGTGAGCACAGGCTGGGCAGCTGTGAGAGTCCAGAGTGGGGTGGCCCTGGGGACGCTTCCAGGCCAGCGGTTCCCTGCACCCCACCAGCTGATTTCGAGCGTGGCAGAGGGAAGGAAAGGGGCGAGCGGGCTGGGCAATGGACCCGACAGGAAACGGGGACTTAGGGGAACACGCTGGAGATGCCATGTGTGGCTGCCGAAGGTCACCATCTCTCCTCAGTGGCTCCCCAGAGCAGGTGCTTTTAAGAACCCTGTTTCCTCTCAGAGCCCAGGGAGAGTCCAAGGACATGGCGCATCAGGAAGTGGGACTGCAGGAGTTCTCTGGTGGCCTCGTGCTGTCCCTCTGGCCACTTCTCACTTTAGGGTGGTCAGCGGCAGCTCGCCATGGCAGTGCCCATTGGTGCACACTAACCTCAGTGGAAAAGTAACCATTCCCTGCCTCTTAGAAAGAACTCATTCTTAGTTTTAGGAGGGTTCCTGTCGCTGAATCAAGTCGCTGCCCTGGATGCAGGGCTGGCCTGGGCGACCCTCCAGGGATGAGGAGCTCAGAATTCCAGTCTTCTAATGTCCACGGACACCTCCCCATCCCTCTAACGTACTGACTATGTCTTTTGATTTAGCATGTCTTCTATAGACCTTCCAAAGAGACCCACACTGGCACTGTCACCCCCTAGGAGGGAAGGTGATGGTTGATGTAGCCCGACGCGCATCTTGTTAATCCGTTCTAATTCCGAGGAGAGTGTGGGTTTAAGATAACACCTATTAATGCATTGCCACAATAATGTGGGGGTAAGAGAAACGCAGGGACGAAACTTCCAGAAACAAACCCTCCAGATCGTTCCACAGGAGTGTTCGCCCTCCGGTGTGACTGAACGACCGACCTTGCCCATGGCTTCATCCAGACAGCACAGCTGCAGTATGGCTGGACAGAAGCACCTACTGTTCTTGGATATTGAAATAAAATAATAAACTTGCAATGATCTTTG"

In [4]:
# complement and reverse DNA or RNA strand
comp = MBP_seq.lower()[::-1].replace("g", "C").replace("c", "G").replace("t", "A").replace("a", "T").replace("u", "A")


In [5]:
# splitting probes
split_probes = []
for n in range(20, 31): 
    for i in range(0, len(comp), n):
        probe = comp[i:i + n]
        
        
        if len(probe) > 20:
            split_probes.append(probe)

# Not sure if we should include file below

# f = open("split_probes.txt", "w")
# f.write(str(split_probes))
# f.close()

PNAS rules:

In [6]:
# PNAS 1
def pnas_filter_1(probe_sequence):
    percent_a = probe_sequence.count('A')/len(probe_sequence)
    return 0.28 < percent_a

In [7]:
# PNAS 2
def pnas_filter_2(probe_sequence):
    A_stack = 'AAAA'
    if A_stack in probe_sequence:
        return False 
    else:
        return True

In [8]:
# PNAS 3
def pnas_filter_3(probe_sequence):
    percent_c = probe_sequence.count('C')/len(probe_sequence)
    return 0.22 < percent_c < 0.28 

In [9]:
# PNAS 4
def pnas_filter_4(probe_sequence):
    C_stack = 'CCCC'
    sequence_subset = probe_sequence[0:12]
    if C_stack in sequence_subset:
        return False
    else:
        return True  

In [10]:
# PNAS 5
def pnas_filter_5(probe_sequence):
    sequence_subset = probe_sequence[0:12]
    for p in range(6):
        if sequence_subset[p:p+6].count('C') >= 4:
            return False
    return True

deltaG:

In [11]:
def deltaG1(probe_sequence):
    import pandas as pd
    
    table = {"AA":0.2, "AC":-1.4, "AG":-0.4, "AT":-0.4,
        "CA":-1.6, "CC":-2.3, "CG":-1.4, "CT":-1.3,
        "GA":-1.4, "GC":-2.0, "GG":-1.7, "GT":-1.5, 
        "TA":-0.5, "TC":-1.5, "TG":-1.2, "TT":-0.7}
    
    for p in range(len(probe_sequence)):
        list_of_pairs = [probe_sequence[p:2 + p] for p in range(0, len(probe_sequence))]
        list_of_pairs.pop()
        if len(list_of_pairs) > 0:
            C = (pd.Series(list_of_pairs)).map(table)
            D = list(C)
            return sum(D)

Percent of GC:

In [12]:
def GC_percent(probe_sequence):
    G = probe_sequence.count('G')
    C = probe_sequence.count('C')
    percent_GC = (G + C)/len(probe_sequence)
    return percent_GC

In [13]:
def GC_filter(probe_sequence):
    return 0.40 <= GC_percent(probe_sequence) <= 0.60

Start and end indexes:

In [14]:
def start_index(probe_sequence):
    if probe in comp:
        start = comp.find(probe_sequence)
        return start

In [15]:
def end_index(probe_sequence):
    if probe in comp:
        end = comp.find(probe_sequence)
        end += len(probe_sequence) - 1
        return end

FLAP

In [18]:
target_name = 'MBP'
FLAP_name = 'X'
FLAP_sequence = 'CCTCCTAAGTTTCGAGCTGGACTCAGTG'
final_sequence = [FLAP_sequence + probe for probe in split_probes]
df_index = df.index.map(str)
final_sequence_name = (target_name + '-' + FLAP_name + '-' + df_index)

In [24]:
# Well position for ordering .xlsx
def well_position_list(n):
    well_y = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    newrow = [[i+str(j) for j in range(1,13)] for i in well_y]
    well_name = [j for i in newrow for j in i]
    well_name_array = np.array(well_name)
    
    well_names_output = []
    for p in range(n):
        well_names_output.append(well_name_array[p % len(well_name_array)])

    return well_names_output

In [25]:
import pandas as pd
import numpy as np

df = pd.DataFrame()
df['probe_sequence'] = split_probes
df['probe_length'] = df['probe_sequence'].apply(len)
df['start'] = df['probe_sequence'].apply(start_index)
df['end'] = df['probe_sequence'].apply(end_index)
df['pnas1'] = df['probe_sequence'].apply(pnas_filter_1)
df['pnas2'] = df['probe_sequence'].apply(pnas_filter_2)
df['pnas3'] = df['probe_sequence'].apply(pnas_filter_3)
df['pnas4'] = df['probe_sequence'].apply(pnas_filter_4)
df['pnas5'] = df['probe_sequence'].apply(pnas_filter_5)
df['deltaG'] = df['probe_sequence'].apply(deltaG1)
df['GC'] = df['probe_sequence'].apply(GC_percent)
df['GC_filter'] = df['probe_sequence'].apply(GC_filter)
df['target_name'] = target_name
df['FLAP_name'] = FLAP_name
df['FLAP_sequence'] = FLAP_sequence
df['final_sequence'] = final_sequence

fdf = iteratively_find_probe_set(df, filters)

fdf_index_reset = fdf.reset_index()
fdf_index = fdf_index_reset.index.map(str)
final_sequence_name = (target_name + '-' + FLAP_name + '-' + fdf_index)

fdf['final_sequence_name'] = final_sequence_name
fdf['well_position'] = well_position_list(len(fdf))
fdf

Found 32 probes using filters pnas3


,probe_sequence,probe_length,start,end,pnas1,pnas2,pnas3,pnas4,pnas5,deltaG,GC,GC_filter,target_name,FLAP_name,FLAP_sequence,final_sequence,final_sequence_name,well_position
380,CCATGGTACTTGCTGTGGCCAAGT,24,2016,2039,False,True,True,True,True,-30.3,0.541667,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGCCATGGTACTTGCTGTGG...,MBP-X-0,A1
841,CTCTTCCTCCCAGCTTAAAGATTTTGGAAA,30,1530,1559,False,True,True,True,False,-30.3,0.400000,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGCTCTTCCTCCCAGCTTAA...,MBP-X-1,A2
537,CAGCTAAATCTGCTGAGGGACAGGCC,26,1664,1689,False,True,True,True,True,-30.3,0.576923,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGCAGCTAAATCTGCTGAGG...,MBP-X-2,A3
759,CAGAGAAGACCCCGAGGAAACTCAATCTT,29,1276,1304,True,True,True,True,True,-30.5,0.482759,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGCAGAGAAGACCCCGAGGA...,MBP-X-3,A4
600,GGGTGCGCCTCGCCGTGAAAAGAAAGT,27,1161,1187,False,False,True,True,False,-30.5,0.592593,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGGGGTGCGCCTCGCCGTGA...,MBP-X-4,A5
512,TCCAAGACTGTCTGATCCTGTTAGGA,26,1014,1039,False,True,True,True,True,-30.7,0.461538,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGTCCAAGACTGTCTGATCC...,MBP-X-5,A6
567,AACAAGATGCGCGTCGGGCTACATCAA,27,270,296,True,True,True,True,True,-30.7,0.518519,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGAACAAGATGCGCGTCGGG...,MBP-X-6,A7
154,GGTGTGCCTGTCTATCCGCAGT,22,1122,1143,False,True,True,True,True,-29.8,0.590909,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGGGTGTGCCTGTCTATCCG...,MBP-X-7,A8
299,CAGCTGTGCTGTCTGGATGAAGCC,24,72,95,False,True,True,True,True,-29.8,0.583333,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGCAGCTGTGCTGTCTGGAT...,MBP-X-8,A9
471,GGTCTGAAGCTCGTCGGACTCTGAG,25,2125,2149,False,True,True,True,True,-31.0,0.600000,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGGGTCTGAAGCTCGTCGGA...,MBP-X-9,A10


Overlap:

In [19]:
def overlapping_intervals(a, b):
    if a[0] < b[0]:
        first = a
        second = b
    elif a[0] == b[0]:
        return True
    else:
        first = b
        second = a
        
    return first[1] > second[0]

In [20]:
def check_probe_fits(probe_candidate, probe_list):
    for probe in probe_list:
        if overlapping_intervals([probe['start'], probe['end']], [probe_candidate['start'], probe_candidate['end']]):
            return False
    return True

def find_nonoverlapping_probes_around_median(df):
    
    # Calculate median delta G
    median = df['deltaG'].median()
    
    # Order rows in probes dataframe by distance to median with Pandas-function
    for probe in range(len(df['deltaG'])):
        result_index = df['deltaG'].sub(median).abs().argsort()[0:].tolist()
        ordered_rows = df.iloc[result_index]

    # Iteratively build list of non-overlapping probes around the median
    final_probe_list = [df.iloc[result_index[0]]]
    for index in result_index[1:]:
        probe_candidate = df.iloc[index]
        if check_probe_fits(probe_candidate, final_probe_list):
            final_probe_list.append(probe_candidate)
    
    return pd.DataFrame(final_probe_list)

In [21]:
# Goal: given a minimum probe set size, find a set of probes that adheres to as many filters as possible

# Start with all five filters on. Take filtered dataframe and generate the probe set using find_nonoverlapping_probes_around_median.
# If the probe set size is too small, relax the least stringent

filters = ['pnas1', 'pnas2', 'pnas3', 'pnas4', 'pnas5']

gc_filter = ['GC_filter']

# Filter for GC content between 40% to 60%
def filter_gc_df(df):
    return df[df[gc_filter].all(True)]

# Given dataframe and filters, return filtered dataframes
def filter_df(df, filters):
    return df[df[filters].all(True)]

# NOTE: Requiers 
def iteratively_find_probe_set(df, filters):
    
    filtered_out_counts = {}

    for f in filters:
        filtered_out_count = len(df) - df[f].sum()
        filtered_out_counts[f] = filtered_out_count

    ordered_filters = sorted(filtered_out_counts, key=filtered_out_counts.get, reverse=False)
    
    gc_filter_on = filter_gc_df(df)
    
    median_and_filters = find_nonoverlapping_probes_around_median(filter_df(gc_filter_on, filters))

    while len(median_and_filters) < 20:
        ordered_filters.pop(0)
        filtered_filters_df = filter_df(gc_filter_on, ordered_filters)
        median_and_filters = find_nonoverlapping_probes_around_median(filter_df(gc_filter_on, ordered_filters))
        
        if len(median_and_filters) >= 20:
            break
    
    if len(median_and_filters) >= 20:
        print('Found {} probes using filters {}'.format(len(median_and_filters), ' '.join(ordered_filters)))
        return median_and_filters
    else:
        raise ValueError('Not enough probes even with all filters off')

In [22]:
iteratively_find_probe_set(df, filters)

Found 32 probes using filters pnas3


,probe_sequence,probe_length,start,end,pnas1,pnas2,pnas3,pnas4,pnas5,deltaG,GC,GC_filter,target_name,FLAP_name,FLAP_sequence,final_sequence
380,CCATGGTACTTGCTGTGGCCAAGT,24,2016,2039,False,True,True,True,True,-30.3,0.541667,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGCCATGGTACTTGCTGTGG...
841,CTCTTCCTCCCAGCTTAAAGATTTTGGAAA,30,1530,1559,False,True,True,True,False,-30.3,0.400000,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGCTCTTCCTCCCAGCTTAA...
537,CAGCTAAATCTGCTGAGGGACAGGCC,26,1664,1689,False,True,True,True,True,-30.3,0.576923,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGCAGCTAAATCTGCTGAGG...
759,CAGAGAAGACCCCGAGGAAACTCAATCTT,29,1276,1304,True,True,True,True,True,-30.5,0.482759,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGCAGAGAAGACCCCGAGGA...
600,GGGTGCGCCTCGCCGTGAAAAGAAAGT,27,1161,1187,False,False,True,True,False,-30.5,0.592593,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGGGGTGCGCCTCGCCGTGA...
512,TCCAAGACTGTCTGATCCTGTTAGGA,26,1014,1039,False,True,True,True,True,-30.7,0.461538,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGTCCAAGACTGTCTGATCC...
567,AACAAGATGCGCGTCGGGCTACATCAA,27,270,296,True,True,True,True,True,-30.7,0.518519,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGAACAAGATGCGCGTCGGG...
154,GGTGTGCCTGTCTATCCGCAGT,22,1122,1143,False,True,True,True,True,-29.8,0.590909,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGGGTGTGCCTGTCTATCCG...
299,CAGCTGTGCTGTCTGGATGAAGCC,24,72,95,False,True,True,True,True,-29.8,0.583333,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGCAGCTGTGCTGTCTGGAT...
471,GGTCTGAAGCTCGTCGGACTCTGAG,25,2125,2149,False,True,True,True,True,-31.0,0.600000,True,MBP,X,CCTCCTAAGTTTCGAGCTGGACTCAGTG,CCTCCTAAGTTTCGAGCTGGACTCAGTGGGTCTGAAGCTCGTCGGA...


<font size="5">Input Version</font>

In [ ]:
# would it be input or sys.arg???

# raw_seq = 'CAAAGATCATTGCAAGTTTATTATTTTATTTCAATATCCAAGAACAGTAGGTGCTTCTGTCCAGCCATACTGCAGCTGTGCTGTCTGGATGAAGCCATGGGCAAGGTCGGTCGTTCAGTCACACCGGAGGGCGAACACTCCTGTGGAACGATCTGGAGGGTTTGTTTCTGGAAGTTTCGTCCCTGCGTTTCTCTTACCCCCACATTATTGTGGCAATGCATTAATAGGTGTTATCTTAAACCCACACTCTCCTCGGAATTAGAA'
# comp = raw_seq.lower()[::-1].replace("g", "C").replace("c", "G").replace("t", "A").replace("a", "T").replace("u", "A")

# if len(comp) < 600:
#     print('Target RNA too short. Minimum length is ___ bases')
#     print('Your target RNA is ' + str(len(raw_seq)) + ' bases long.')
# else:
#     pass